In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import csv
import chromedriver_binary
import urllib
import os
import requests
import json

In [ ]:
def get_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    return webdriver.Chrome( options = options )

def get_page_bs( driver, url ):
    driver.get( url )
    time.sleep( 5 ) # 5秒間待機
    return BeautifulSoup( driver.page_source, features='lxml' )
        
def write_csv( file_path, data ):
    with open( file_path, 'a' ) as f:
        print( ','.join( data ), file=f )
        
def read_csv( file_path ):
    with open( file_path ) as f:
        reader = csv.reader(f)
        return [row for row in reader]
        
def r( string ):
    return string.replace("\n","").replace("\u3000"," ")

def get_seller_url( seller_id ):
    return 'https://www.amazon.co.jp/sp?is-secure=true&&ie=UTF8&seller=' + seller_id

# これより上はテンプレート


def get_seller_detail( driver, seller_id, url ):
    #url =get_seller_url( seller_id )
    print( url + " を検索中です" )
    bs = get_page_bs( driver, url )
    datas = bs.find_all( 'div', class_ = 'a-row a-spacing-medium' )
    if len( datas ) < 2:
        return ("")
    s = datas[1].text
    ind1 = s.find("販売業者")
    ind2 = s.find("お問い合わせ先電話番号")
    ind3 = s.find("住所")
    ind4 = s.find("運営責任者名")
    ind5 = s.find("店舗名")
    p1 = ""
    p2 = ""
    if len(  bs.select('.feedback-detail-description') ) > 0:
        point = bs.select('.feedback-detail-description')[0].text
        p_ind1 = point.find("過去12ヵ月間で")
        p_ind2 = point.find("評価：")
        p1 = point[ : p_ind2 - 2 ]
        p2 = point[ p_ind2 + 3: len( point ) - 1 ]
    return [
            p1,
            p2,
            r(s[ind1+5:ind2].replace('\n','')),
            r(s[ind2+12:ind3].replace('\n','')),
            r(s[ind3+3:ind4].replace('\n','')),
            r(s[ind4+7:ind5].replace('\n','')),
            r(s[ind5+4:].replace('\n',''))
    ]

d = get_driver()
# print( get_seller_detail( d, "AE6MAGS149M00"))
datas = read_csv("../seikatsunoki/seller.csv")

FILE_NAME = "output/p001_seikatsunoki.csv"
for i in range(0,len(datas)):
    print( str(i)+"行目を検索中" )
    output = datas[i]
    if len(datas[i]) > 2:
        #seller_id = datas[i][1][datas[i][1].find('seller=')+7:datas[i][1].find('seller=')+21]
        #rint( seller_id )
        seller_detail = get_seller_detail( d, datas[i][2], datas[i][1] )
        
        output.extend( seller_detail )
        print( output )
    write_csv( FILE_NAME, output )